In [1]:
import torch
from torch.utils.data import random_split, DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

import pickle
import base64
import sys

import hashlib
import json
from time import time
from time import sleep
import requests

%matplotlib inline
plt.rcParams['figure.figsize'] = [5, 5]

In [2]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 5.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [3]:
from torch import tensor
from torchmetrics.classification import MulticlassRecall
from torchmetrics.classification import MulticlassPrecision

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
classes = 10
input_dim = 784

batch_size = 128
epochs_per_client = 3
learning_rate = 2e-2

In [6]:
# CONFIGURE BEFORE RUNNIG
# MAKE SURE SAME FOR BOTH SERVER AND ALL CLIENTS' CODE
num_clients = 2
rounds = 2

In [7]:
# CONFIGURE CLIENT ID TO CONTAIN ALL POSSIBLE CLIENT IDs
# Ex. If there are 3 clients, the only possible client ids are 0, 1, 2
# The client_id variable will therefore be equal to [0,1,2]
client_id = [0,1]

In [8]:
client_code = [f'L{client_id[i]+1}' for i in client_id]

In [9]:
# Make sure there is no '/' at the end of the url.
replit_url = "https://8e7399ac-6fda-47b4-9b7b-e7a2ed8d5bf1-00-1vdkonhfb818n.riker.replit.dev"

CHAIN_URL = f'{replit_url}/chain'
MINE_URL = f'{replit_url}/mine'
SEND_TRNS_URL = f'{replit_url}/transactions/new'
GET_TRNS_URL = f'{replit_url}/current'
PREV_BLK_URL = f'{replit_url}/last'

In [10]:
def get_device():
    return torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader(DataLoader):
        def __init__(self, dl, device):
            self.dl = dl
            self.device = device

        def __iter__(self):
            for batch in self.dl:
                yield to_device(batch, self.device)

        def __len__(self):
            return len(self.dl)

device = get_device()

In [11]:
class FederatedNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(1, 20, 7)
        self.conv2 = torch.nn.Conv2d(20, 40, 7)
        self.maxpool = torch.nn.MaxPool2d(2, 2)
        self.flatten = torch.nn.Flatten()
        self.linear = torch.nn.Linear(2560, 10)
        self.non_linearity = torch.nn.functional.relu
        self.track_layers = {'conv1': self.conv1, 'conv2': self.conv2, 'linear': self.linear}

    def forward(self, x_batch):
        out = self.conv1(x_batch)
        out = self.non_linearity(out)
        out = self.conv2(out)
        out = self.non_linearity(out)
        out = self.maxpool(out)
        out = self.flatten(out)
        out = self.linear(out)
        return out

    def get_track_layers(self):
        return self.track_layers

    def apply_parameters(self, parameters_dict):
        with torch.no_grad():
            for layer_name in parameters_dict:
                self.track_layers[layer_name].weight.data *= 0
                self.track_layers[layer_name].bias.data *= 0
                self.track_layers[layer_name].weight.data += parameters_dict[layer_name]['weight']
                self.track_layers[layer_name].bias.data += parameters_dict[layer_name]['bias']

    def get_parameters(self):
        parameters_dict = dict()
        for layer_name in self.track_layers:
            parameters_dict[layer_name] = {
                'weight': self.track_layers[layer_name].weight.data,
                'bias': self.track_layers[layer_name].bias.data
            }
        return parameters_dict

    def batch_accuracy(self, outputs, labels):
        with torch.no_grad():
            _, predictions = torch.max(outputs, dim=1)
            return torch.tensor(torch.sum(predictions == labels).item() / len(predictions))

    def _process_batch(self, batch):
        images, labels = batch
        outputs = self(images)
        loss = torch.nn.functional.cross_entropy(outputs, labels)
        accuracy = self.batch_accuracy(outputs, labels)
        precision = MulticlassPrecision(num_classes=10).to(device)
        precision_score = precision(outputs, labels)
        recall = MulticlassRecall(num_classes=10).to(device)
        recall_score = recall(outputs, labels)
        return (loss, accuracy, precision_score, recall_score)

    def fit(self, dataset, epochs, lr, batch_size=128, opt=torch.optim.SGD):
        dataloader = DeviceDataLoader(DataLoader(dataset, batch_size, shuffle=True), device)
        optimizer = opt(self.parameters(), lr)
        history = []
        for epoch in range(epochs):
            losses = []
            accs = []
            prec_list = []
            rec_list = []
            for batch in dataloader:
                loss, acc, prec, rec = self._process_batch(batch)
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                loss.detach()
                losses.append(loss)
                accs.append(acc)
                prec_list.append(prec)
                rec_list.append(rec)
            avg_loss = torch.stack(losses).mean().item()
            avg_acc = torch.stack(accs).mean().item()
            avg_prec = torch.stack(prec_list).mean().item()
            avg_rec = torch.stack(rec_list).mean().item()
            f1 = (2*avg_prec*avg_rec) / (avg_prec + avg_rec)
            history.append((avg_loss, avg_acc, avg_prec, avg_rec, f1))
        return history

    def evaluate(self, dataset, batch_size=128):
        dataloader = DeviceDataLoader(DataLoader(dataset, batch_size), device)
        losses = []
        accs = []
        prec_list = []
        rec_list = []
        with torch.no_grad():
            for batch in dataloader:
                loss, acc, prec, rec = self._process_batch(batch)
                losses.append(loss)
                accs.append(acc)
                prec_list.append(prec)
                rec_list.append(rec)
        avg_loss = torch.stack(losses).mean().item()
        avg_acc = torch.stack(accs).mean().item()
        avg_prec = torch.stack(prec_list).mean().item()
        avg_rec = torch.stack(rec_list).mean().item()
        f1 = (2*avg_prec*avg_rec) / (avg_prec + avg_rec)
        return (avg_loss, avg_acc, avg_prec, avg_rec, f1)

In [12]:
class Client:
    def __init__(self, client_id, dataset):
        self.client_id = client_id
        self.dataset = dataset

    def get_dataset_size(self):
        return len(self.dataset)

    def get_client_id(self):
        return self.client_id

    def train(self, parameters_dict):
        net = to_device(FederatedNet(), device)
        net.apply_parameters(parameters_dict)
        train_history = net.fit(self.dataset, epochs_per_client, learning_rate, batch_size)
        print('{}: Loss = {}, Accuracy = {}, Precision = {}, Recall = {}, F1 = {}'.format(self.client_id, round(train_history[-1][0], 4), round(train_history[-1][1], 4), round(train_history[-1][2], 4), round(train_history[-1][3], 4), round(train_history[-1][4], 4)))
        return net.get_parameters()

In [13]:
def compress_params(params):
  compressed = pickle.dumps(params)
  params_bytes = base64.b64encode(compressed)
  params_bystr = params_bytes.decode('ascii')
  return params_bystr

def decompress_params(params):
  decompressed = params.encode("ascii")
  decompressed = base64.b64decode(decompressed)
  decompressed = pickle.loads(decompressed)
  return decompressed

In [14]:
client_datasets = []

for i in range(8):
  with open(f'/content/gdrive/MyDrive/Colab Notebooks/BC+FL Code/datasets/client_train_set_{i}.pickle', 'rb') as fp:
    client_datasets.append(pickle.load(fp))

In [15]:
clients = []

for i in client_id:
  client_dataset = None
  if int(8 / num_clients == 1):
    client_dataset = client_datasets[client_id]
  else:
    sets_per_client = int(8 / num_clients)
    # for i in range(sets_per_client):
    #   print(sets_per_client * client_id + i)
    start = sets_per_client * client_id[i]
    end = sets_per_client * client_id[i] + sets_per_client
    client_dataset = torch.utils.data.ConcatDataset(client_datasets[start:end])
  clients.append(Client('client_' + str(i), client_dataset))

In [16]:
prev_block_index = -1
for i in range(rounds):
  response = requests.get(PREV_BLK_URL)
  block_index = response.json()['chain']['index']

  # print(prev_block_index)
  # print(block_index)

  # if i == 0:
  #   prev_block_index = block_index

  # print(prev_block_index)

  new_block = False if int(prev_block_index) == int(block_index) else True
  # print(new_block)
  # print(response.json()['chain']['transactions'][-2]['type'] != 'global')
  cond = response.json()['chain']['transactions'][-2]['type'] != 'global' or new_block == False
  print(cond)

  while response.json()['chain']['transactions'][-2]['type'] != 'global' or new_block == False:
    sleep(30)
    response = requests.get(PREV_BLK_URL)
    block_index = response.json()['chain']['index']
    new_block = False if int(prev_block_index) == int(block_index) else True
    print(cond)

  prev_block_index = block_index

  curr_global_gradients = response.json()['chain']['transactions'][-2]['gradients']
  curr_global_gradients = decompress_params(curr_global_gradients)

  for layer_name in curr_global_gradients:
    curr_global_gradients[layer_name]['weight'] = curr_global_gradients[layer_name]['weight'].to(device)
    curr_global_gradients[layer_name]['bias'] = curr_global_gradients[layer_name]['bias'].to(device)

  for j in client_id:
    sleep(5)
    client_parameters = clients[j].train(curr_global_gradients)

    for layer_name in client_parameters:
      client_parameters[layer_name]['weight'] = client_parameters[layer_name]['weight'].to('cpu')
      client_parameters[layer_name]['bias'] = client_parameters[layer_name]['bias'].to('cpu')

    client_data_size = clients[j].get_dataset_size()
    client_parameters_compressed = compress_params(client_parameters)
    new_transaction = {
        'type': 'local',
        'trainer': client_code[j],
        'gradient': client_parameters_compressed,
        'data-size': client_data_size
    }

    # Send the transaction to the server
    response = requests.post(SEND_TRNS_URL, json=new_transaction)
    print(response.json())

False
client_0: Loss = 0.1993, Accuracy = 0.9409, Precision = 0.9413, Recall = 0.9407, F1 = 0.941
{'message': 'Transaction will be added to Block 3'}
client_1: Loss = 0.2044, Accuracy = 0.9422, Precision = 0.9433, Recall = 0.9421, F1 = 0.9427
{'message': 'Transaction will be added to Block 3'}
True
True
client_0: Loss = 0.1125, Accuracy = 0.9674, Precision = 0.9676, Recall = 0.9673, F1 = 0.9675
{'message': 'Transaction will be added to Block 5'}
client_1: Loss = 0.1131, Accuracy = 0.9669, Precision = 0.9669, Recall = 0.9666, F1 = 0.9668
{'message': 'Transaction will be added to Block 5'}


In [17]:
print(response)

<Response [201]>


In [18]:
global_net = to_device(FederatedNet(), 'cpu')
curr_parameters = global_net.get_parameters()

for layer_name in curr_parameters:
    curr_parameters[layer_name]['weight'] = curr_parameters[layer_name]['weight'].to(device)
    curr_parameters[layer_name]['bias'] = curr_parameters[layer_name]['bias'].to(device)

for j in client_id:
  #clients[j].to(device)
  client_parameters = clients[j].train(curr_parameters)
  client_data_size = clients[j].get_dataset_size()
  print(client_data_size)

client_0: Loss = 0.1968, Accuracy = 0.9422, Precision = 0.9422, Recall = 0.9415, F1 = 0.9419
24900
client_1: Loss = 0.2031, Accuracy = 0.9416, Precision = 0.9428, Recall = 0.9407, F1 = 0.9417
24900
